In [1]:
import math
from sklearn.cross_validation import KFold
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
import pickle
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.grid_search import GridSearchCV

In [2]:
%run 'methods.ipynb'
%run 'XGBoost_class.ipynb'

In [3]:
df_train = pd.read_csv('train.csv')
labels, y = getLabeled(df_train['TripType'])


num_classes = len(np.unique(y))

params = {    
              'subsample':0.8,
              'colsample_bytree': 0.7,
              'objective': 'multi:softprob',
              'eval_metric': 'mlogloss',
              'num_class': num_classes,
              'silent':1,
              'eta': 0.3,
              'max_depth': 8,
              'min_child_weight': 1,
              'num_round': 1024
}

#k fold validation
kf= KFold(len(y), n_folds=10, shuffle=True, random_state=None)

#for num_round in [1024]:
for eta in [0.03, 0.06, 0.1]:
    for max_depth in [4, 5, 6, 7, 8]:
        avg = 0
        for train_index, test_index in kf:
            X_train, X_test = df_train.iloc[train_index] , df_train.iloc[test_index] 
                
            features_train, features_test = generateTestTrainFeature(X_train, X_test)
                    
            #features_train = getFeaturesTrain(X_train)
            #vec_desc, features_train = processDescTrain(X_train, True)
            #upc_dict, upc_vec, upc_feature = getCountVectorTrain(X_train, 'Upc')
            #fileno_dict, fileno_vec, fileno_feature = getCountVectorTrain(X_train, 'FinelineNumber')
                    
            #features_test = getFeaturesTest(X_test)
            #features_test = processDescTest(X_test, True, vec_desc)
            #upc_feature_test = getCountVectorTest(X_test, 'Upc', upc_dict, upc_vec)
            #fileno_feature_test = getCountVectorTest(X_test, 'FinelineNumber', fileno_dict, fileno_vec)
                    
            y_train = getY(X_train, labels)
            y_test = getY(X_test, labels)
                            
            
            params['eta'] = eta
            params['max_depth'] = max_depth
                
            clfxgb = XGBoostClassifier(**params)
            clfxgb.fit(features_train , y_train )
            y_pred = clfxgb.predict_proba(features_test)
            avg += loss_function(y_test, y_pred)

        print eta, max_depth,"=>", avg/len(kf)

0.03 4 => 3.32202348104
0.03 5 => 3.31729984877
0.03 6 => 3.31164573072
0.03 7 => 3.30799609555
0.03 8 => 3.3041840104
0.06 4 => 3.22628921903
0.06 5 => 3.22593791766
0.06 6 => 3.22214575574
0.06 7 => 3.2198899137
0.06 8 => 3.21885637876
0.1 4 => 3.19964850381
0.1 5 => 3.20592450631
0.1 6 => 3.20803312796
0.1 7 => 3.21135880275
0.1 8 => 3.21306480344


/Users/atulkumar/anaconda/lib/python2.7/site-packages/pandas/core/generic.py:2273: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [ ]:
df_test = pd.read_csv('test.csv')
features_train, features_test = generateTestTrainFeature(df_train, df_test)

y_train = getY(df_train, labels)

params['eta'] = 0.1
params['max_depth'] = 8
                
clfxgb = XGBoostClassifier(**params)
clfxgb.fit(features_train , y_train )

In [ ]:
generateSubmission(features_test, clfxgb, labels, "xgb_all")